<center>
<img src="../../img/ods_stickers.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/). Translated and edited by [Sergey Isaev](https://www.linkedin.com/in/isvforall/), [Artem Trunov](https://www.linkedin.com/in/datamove/), [Anastasia Manokhina](https://www.linkedin.com/in/anastasiamanokhina/), and [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

Unique values of all features (for more information, please see the links above):
- `age`: continuous.
- `workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- `fnlwgt`: continuous.
- `education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- `education-num`: continuous.
- `marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- `occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- `relationship`: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- `race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- `sex`: Female, Male.
- `capital-gain`: continuous.
- `capital-loss`: continuous.
- `hours-per-week`: continuous.
- `native-country`: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.   
- `salary`: >50K,<=50K

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
# to draw pictures in jupyter notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../adult.data.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**1. How many men and women (*sex* feature) are represented in this dataset?** 

In [3]:
data.sex.value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

**2. What is the average age (*age* feature) of women?**

In [36]:
average_women = data.loc[data.sex == 'Female'].age.mean()
average_women

36.85823043357163

**3. What is the percentage of German citizens (*native-country* feature)?**

In [37]:
germans = data.loc[data['native-country'] == 'Germany', 'native-country'].count()
germans

137

**4-5. What are the mean and standard deviation of age for those who earn more than 50K per year (*salary* feature) and those who earn less than 50K per year? **

In [7]:
slrs_more = data.loc[data.salary.isin(['>50K']), ['age']]
slrs_less = data.loc[data.salary.isin(['<50K']), ['age']]
print("Age - more than 50K per year\n")
print(f"mean {slrs_more.mean()} \nstd dev {slrs_more.std()}\n")
print("Age - less than 50K per year\n")
print(f"mean {slrs_less.mean()} \nstd dev {slrs_less.std()}")

Age - more than 50K per year

mean age    44.249841
dtype: float64 
std dev age    10.519028
dtype: float64

Age - less than 50K per year

mean age   NaN
dtype: float64 
std dev age   NaN
dtype: float64


**6. Is it true that people who earn more than 50K have at least high school education? (*education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters* or *Doctorate* feature)**

In [16]:
# me imagino que debo buscar personas con >50K que estén en las categorías
# Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters or Doctorate
# Entonces buscaré las personas dentro de esta categoría, luego de las 
# categorías que faltan, hare una resta del numero de personas de ambas categorias.
# Devuelvo True si no hay gente que tenga >50K y no esté en las categorías provistas.
par = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']
eductn_positive = data.loc[data.education.isin(par) & data.salary.isin(['>50K']),
    ['education']].count()
eductn_negative = data.loc[(~data.education.isin(par)) & data.salary.isin(['>50K']),
    ['education']].count()
(eductn_positive - eductn_negative) == eductn_positive

education    False
dtype: bool

**7. Display age statistics for each race (*race* feature) and each gender (*sex* feature). Use *groupby()* and *describe()*. Find the maximum age of men of *Amer-Indian-Eskimo* race.**

In [17]:
groupby_Race = data.groupby('race').describe()
print('===============================================================',
      '\n   groupby_Race',
      '\n===============================================================')
print(groupby_Race)
groupby_Sex = data.groupby('sex').describe()
print('\n===============================================================',
      '\n   groupby_Sex',
      '\n===============================================================')
print(groupby_Sex)
print('\n===============================================================',
      '\n   maximum age of men of Amer-Indian-Eskimo race',
      '\n===============================================================')
max_age_AIE = data.loc[data.race.isin(['Amer-Indian-Eskimo']), ['age']].max()
print(max_age_AIE)

   groupby_Race 
                        age                                                \
                      count       mean        std   min   25%   50%   75%   
race                                                                        
Amer-Indian-Eskimo    311.0  37.173633  12.447130  17.0  28.0  35.0  45.5   
Asian-Pac-Islander   1039.0  37.746872  12.825133  17.0  28.0  36.0  45.0   
Black                3124.0  37.767926  12.759290  17.0  28.0  36.0  46.0   
Other                 271.0  33.457565  11.538865  17.0  25.0  31.0  41.0   
White               27816.0  38.769881  13.782306  17.0  28.0  37.0  48.0   

                           fnlwgt                                         \
                     max    count           mean            std      min   
race                                                                       
Amer-Indian-Eskimo  82.0    311.0  120831.147910   88338.636336  12285.0   
Asian-Pac-Islander  90.0   1039.0  159940.609240   85122.30750

**8. Among whom is the proportion of those who earn a lot (>50K) greater: married or single men (*marital-status* feature)? Consider as married those who have a *marital-status* starting with *Married* (Married-civ-spouse, Married-spouse-absent or Married-AF-spouse), the rest are considered bachelors.**

In [18]:
cats = ['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse']
married_men = data.loc[
    (data['marital-status'].isin(cats) & data.salary.isin(['>50K'])), 
    ['marital-status']].count()
single_men = data.loc[
    ((~data['marital-status'].isin(cats)) & data.salary.isin(['>50K'])), 
    ['marital-status']].count()
is_it_true = (married_men - single_men) > 0

print(f"True that married_men earn >50K\n: {is_it_true}")

True that married_men earn >50K
: marital-status    True
dtype: bool


**9. What is the maximum number of hours a person works per week (*hours-per-week* feature)? How many people work such a number of hours, and what is the percentage of those who earn a lot (>50K) among them?**

In [19]:
hpw = data['hours-per-week'].max()
ppl_max_hpw = data.loc[data['hours-per-week'].isin([99]), ['hours-per-week']].count()
ppl_max_hpw_50k = data.loc[
    (data['hours-per-week'].isin([99]) & data.salary.isin(['>50K'])),
    ['hours-per-week']].count()
percent = ppl_max_hpw_50k * 100 / ppl_max_hpw
print(f"maximum number of hours a person works per week\n: {hpw}")
print(f"people work such a number of hours\n: {ppl_max_hpw}")
print(f"what is the percentage of those who earn a lot\n: {percent}")

maximum number of hours a person works per week
: 99
people work such a number of hours
: hours-per-week    85
dtype: int64
what is the percentage of those who earn a lot
: hours-per-week    29.411765
dtype: float64


**10. Count the average time of work (*hours-per-week*) for those who earn a little and a lot (*salary*) for each country (*native-country*). What will these be for Japan?**

In [20]:
a_little = data.loc[
    data.salary.isin(['<=50K']),
    ['hours-per-week','native-country']]
a_little.groupby('native-country').sum()

,hours-per-week
native-country,
?,17552
Cambodia,497
Canada,3109
China,2056
Columbia,2205
Cuba,2659
Dominican-Republic,2879
Ecuador,913
El-Salvador,3495


In [21]:
a_lot = data.loc[
    data.salary.isin(['>50K']),
    ['hours-per-week','native-country']]
a_lot.groupby('native-country').sum()

,hours-per-week
native-country,
?,6650
Cambodia,280
Canada,1780
China,778
Columbia,100
Cuba,1061
Dominican-Republic,94
Ecuador,195
El-Salvador,405


In [22]:
# Those who work a lot Japan 
j_a_lot = a_lot.loc[a_lot['native-country'].isin(['Japan'])]['hours-per-week'].sum()
j_a_lot


1151

In [23]:
# Those who work a little Japan 
j_a_little = a_little.loc[a_little['native-country'].isin(['Japan'])]['hours-per-week'].sum()
j_a_little

1558

In [24]:
# :)